<a href="https://colab.research.google.com/github/AngeMervaneJdev/ML_AI_1/blob/main/tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.3 MB 41 kB/s 
     |████████████████████████████████| 50 kB 3.6 MB/s 
     |████████████████████████████████| 3.1 MB 21.7 MB/s 
     |████████████████████████████████| 488 kB 41.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
      Successfully uninstalled tensorflow-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.preprocessing import OneHotEncoder


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
from keras.datasets import mnist

Using TensorFlow backend.


In [5]:
print(tf. __version__)

1.15.2


#[Problème 1] Retour sur le scratch

Les étapes utiliser dans le scratch sont: 
- 1 - Initialisation du poids
- 2 - utiliser une booucle d'epoque pour 
- 3 - Multiplier le poids par l'entrée
- 4 - appliquer une fonction d'activation
- 5 - extimer et mettre à jour les poids

Mise e oeuvre avec tensorflow
- Initialisation des poids :
- Entrainement avec la boucle epoch

```
for epoch in range(1000):
    sess.run(train_step, feed_dict={
        x:x_train,
        t:y_train
    })
```
- Fonction d'activation
```
y = tf.sigmoid(tf.matmul(x, W) + b)
```
- Evaluation 
```
acc_val = sess.run(accuracy,
     feed_dict={
        x:x_train,
        t:y_train
    })
```

 

#[Problème 2] Considérez la correspondance entre le scratch et TensorFlow

In [8]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [9]:
"""
Classification binaire de l'ensemble de données Iris à l'aide d'un réseau de neurones implémenté dans TensorFlow

"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
tf.test.gpu_device_name() 
"""
tensorflowのバージョンを1.x系に変更した際は忘れずに
「!pip install tensorflow-gpu==1.14.0」でGPUのインストールをしておきましょう。
tf.test.gpu_device_name()でGPUの設定状態を確認し、認識されるかを確認します。
成功している場合はログが出力されます、認識されない場合は何も出力されません。
"""

# データセットの読み込み
df = pd.read_csv("Iris.csv")

# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# ラベルを数値に変換
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)



# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder(dtype="float", shape=[None, n_input])
Y = tf.placeholder(dtype="float", shape=[None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Epoch 0, loss : 5.4193, val_loss : 57.2231, acc : 0.375
Epoch 1, loss : 3.5689, val_loss : 34.6412, acc : 0.375
Epoch 2, loss : 2.1442, val_loss : 15.9005, acc : 0.188
Epoch 3, loss : 1.7802, val_loss : 11.7099, acc : 0.062
Epoch 4, loss : 1.4782, val_loss : 11.4397, acc : 0.188
Epoch 5, loss : 1.2006, val_loss : 12.0541, acc : 0.250
Epoch 6, loss : 1.0606, val_loss : 8.0987, acc : 0.188
Epoch 7, loss : 0.9051, val_loss : 5.9181, acc : 0.312
Epoch 8, loss : 0.7778, val_loss : 5.5638, acc : 0.312
Epoch 9, loss : 0.6314, val_loss : 4.3611, acc : 0.312
Epoch 10, loss : 0.4857, val_loss : 2.7496, acc : 0.438
Epoch 11, loss : 0.3648, val_loss : 2.1538, acc : 0.438
Epoch 12, loss : 0.2902, val_loss : 1.5546, acc : 0.438
Epoch 13, loss : 0.2336, val_loss : 1.0378, acc : 0.562
Epoch 14, loss : 0.1847, val_loss : 0.9605, acc : 0.688
Epoch 15, loss : 0.1568, val_loss : 0.7830, acc : 0.688
Epoch 16, loss : 0.1283, val_loss : 0.5272, acc : 0.812
Epoch 17, loss : 0.1076, val_loss : 0.3977, acc : 0.

#[Problème 3] Créez un modèle d'Iris en utilisant les trois variables objectives.

In [17]:
df = pd.read_csv("Iris.csv")
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# NumPy 
X = np.array(X)
y = np.array(y)
# 
y[y == 'Iris-setosa'] = 0
y[y == "Iris-versicolor"] = 1
y[y == "Iris-virginica"] = 2

y = y.astype(np.int64)[:, np.newaxis]

#one-hotエンコーディング
enc = OneHotEncoder(categories='auto')
y = enc.fit_transform(y).A

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 
X = tf.placeholder(dtype="float", shape=[None, n_input])
Y = tf.placeholder(dtype="float", shape=[None, n_classes])

# train
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):

    tf.random.set_random_seed(0)
    # 
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: X_train, Y: y_train})
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss

        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Epoch 0, loss : 22.1617, val_loss : 150.2520, acc : 0.625
Epoch 1, loss : 9.8478, val_loss : 32.4296, acc : 0.500
Epoch 2, loss : 4.5410, val_loss : 19.4940, acc : 0.583
Epoch 3, loss : 3.2280, val_loss : 15.0229, acc : 0.458
Epoch 4, loss : 2.8753, val_loss : 11.2347, acc : 0.417
Epoch 5, loss : 2.3603, val_loss : 9.9260, acc : 0.417
Epoch 6, loss : 1.9581, val_loss : 8.8065, acc : 0.417
Epoch 7, loss : 1.5335, val_loss : 7.0626, acc : 0.458
Epoch 8, loss : 1.0305, val_loss : 4.8160, acc : 0.500
Epoch 9, loss : 0.4991, val_loss : 1.6103, acc : 0.750
Epoch 10, loss : 0.1910, val_loss : 0.6418, acc : 0.750
Epoch 11, loss : 0.1086, val_loss : 0.6254, acc : 0.875
Epoch 12, loss : 0.0648, val_loss : 0.6366, acc : 0.875
Epoch 13, loss : 0.0414, val_loss : 0.6748, acc : 0.917
Epoch 14, loss : 0.0245, val_loss : 0.9613, acc : 0.833
Epoch 15, loss : 0.0173, val_loss : 1.1632, acc : 0.833
Epoch 16, loss : 0.0122, val_loss : 1.1324, acc : 0.833
Epoch 17, loss : 0.0090, val_loss : 1.0284, acc : 0

#[Problème 4] Création d'un modèle de prix des logements

In [16]:
df = pd.read_csv("train.csv")
y = df['SalePrice']
X = df.loc[:, ['GrLivArea', 'YearBuilt']]

# NumPy 
X = np.array(X)
y = np.array(y)
# 
y = y.astype(np.int64)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.005
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 
X = tf.placeholder(dtype="float", shape=[None, n_input])
Y = tf.placeholder(dtype="float", shape=[None, n_classes])

# train
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):

    tf.random.set_random_seed(0)
    # 
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

                            
logits = example_net(X)

loss_op = tf.reduce_mean(tf.square(Y - logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

pred = logits

mse = tf.reduce_mean(tf.square(Y - pred))
r2 =  1 - (tf.reduce_sum(tf.square(Y - pred)) / tf.reduce_sum(tf.square(Y - tf.reduce_mean(Y))))

init = tf.global_variables_initializer()

# 
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})

            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss

        total_loss /= n_samples
        val_loss= sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    
    #estimation   
    test_mse = sess.run(mse, feed_dict={X: X_test, Y: y_test})
    test_r2 = sess.run(r2, feed_dict={X: X_test, Y: y_test})

    print('test_mse : {:.3f}'.format(test_mse))
    print('test_R2 : {:.3f}'.format(test_r2))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Epoch 0, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 1, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 2, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 3, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 4, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 5, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 6, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 7, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 8, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 9, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 10, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 11, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 12, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 13, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 14, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 15, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 16, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Epoch 17, loss : 0.0000, val_loss : 0.0000, acc : 0.917
Ep

#[Problème 5] Création d'un modèle MNIST

In [19]:
#データの読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#次元変換
y_train = y_train.astype(np.int)[:, np.newaxis]
y_test = y_test.astype(np.int)[:, np.newaxis]

#one-hotエンコーディング
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)

#データの変換
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

#分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

#ハイパーパラメータの設定
learning_rate = 0.01 
batch_size = 100 
num_epochs = 10 
n_hidden1 = 50 
n_hidden2 = 25 
n_input = X_train.shape[1] 
n_samples = X_train.shape[0] 
n_classes = 10 

#計算グラフに渡す引数の形を決める
X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, n_classes])

#trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


def example_net(x):
    '''
    単純な3層ニューラルネットワーク
    '''
    #重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    #全結合層
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] #addと同じ
    
    return layer_output

11493376/11490434 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Depr

In [20]:
#ネットワーク構造の読み込み
logits = example_net(X)

#目的関数
#目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

#最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1))

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


init = tf.global_variables_initializer()

# 
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})

            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc

        total_loss /= total_batch
        total_acc /= total_batch
        val_loss= sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    
    #estimation   
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print('test_acc : {:.3f}'.format(test_acc))

Epoch 0, loss : 5.2121, val_loss : 1.7513, acc : 0.460
Epoch 1, loss : 1.4545, val_loss : 1.2738, acc : 0.630
Epoch 2, loss : 1.0644, val_loss : 1.0179, acc : 0.670
Epoch 3, loss : 0.8634, val_loss : 0.8603, acc : 0.720
Epoch 4, loss : 0.7368, val_loss : 0.7177, acc : 0.750
Epoch 5, loss : 0.6206, val_loss : 0.5989, acc : 0.770
Epoch 6, loss : 0.5194, val_loss : 0.5303, acc : 0.860
Epoch 7, loss : 0.4429, val_loss : 0.4761, acc : 0.920
Epoch 8, loss : 0.3647, val_loss : 0.3705, acc : 0.980
Epoch 9, loss : 0.2823, val_loss : 0.3151, acc : 0.970
test_acc : 0.923
